In [1]:
import numpy as np
import torch
import random
import os
import sys
import scipy.sparse as sp
from sklearn.preprocessing import normalize
import networkx as nx
from sortedcontainers import SortedList
import os
import numpy as np
from torch.utils.data import Dataset
import random
import sys
import time
import json
import torch
import torch.nn as nn
import scipy.sparse as sp
import numpy as np
from utils import get_sparse_tensor, graph_rank_nodes, generate_daj_mat, generate_aug_daj_mat
from torch.nn.init import kaiming_uniform_, xavier_normal, normal_, zeros_, ones_
import sys
import torch.nn.functional as F
from sklearn.preprocessing import normalize
from torch.utils.checkpoint import checkpoint
import dgl
import multiprocessing as mp
import time
from info_nce import InfoNCE
from sklearn.metrics.pairwise import cosine_similarity
from torchmetrics.functional import pairwise_cosine_similarity
import torch
import sys
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam, SGD
import time
import numpy as np
import os
from utils import AverageMeter, get_sparse_tensor
import torch.nn.functional as F
import scipy.sparse as sp
from dataset import AuxiliaryDataset
import csv

In [2]:
from dataset import get_dataset
from model import get_model
from trainer import get_trainer
import torch
from utils import init_run
from tensorboardX import SummaryWriter
from config import get_gowalla_config, get_yelp_config, get_amazon_config

In [3]:
device = torch.device('cuda')
config = get_gowalla_config(device)
dataset_config, model_config, trainer_config = config[10]
dataset_config['path'] = dataset_config['path'][:-4] + str(1)
model_config['name'] = 'DOSE_aug'


In [5]:
#writer = SummaryWriter(log_path)
dataset = get_dataset(dataset_config) # datasetのクラス
model = get_model(model_config, dataset) # modelのクラウス

{'name': 'ProcessedDataset', 'path': 'data/Gowalla/1', 'device': device(type='cuda'), 'neg_ratio': 4}
init dataset ProcessedDataset
{'name': 'DOSE_aug', 'embedding_size': 64, 'n_layers': 3, 'device': device(type='cuda'), 'dropout': 0.3, 'feature_ratio': 1, 'aug_num': 500000, 'dataset': <dataset.ProcessedDataset object at 0x14c18db59d00>}
611909852


In [13]:
#dataset.train_array

In [14]:
"""
class DOSE(BasicModel):
    def __init__(self, model_config):
        super(DOSE, self).__init__(model_config)
        self.embedding_size = model_config['embedding_size']
        self.n_layers = model_config['n_layers']
        self.dropout = model_config['dropout']
        self.feature_ratio = model_config['feature_ratio']  # Template%
        self.norm_adj = self.generate_graph(model_config['dataset'])
       
        self.alpha = 1.
        self.delta = model_config.get('delta', 0.99)
        self.taugh = model_config.get('taugh', 0.2)
        self.aug_num = model_config['aug_num']
        # self.temper = model_config['temper']
        self.feat_mat, self.user_map, self.item_map, self.row_sum = \
            self.generate_feat(model_config['dataset'],
                               ranking_metric=model_config.get('ranking_metric', 'sort'))
        self.update_feat_mat()
        # self.norm_aug_adj = enerate_aug_graph(model_config['dataset'] )
        self.embedding = nn.Embedding(self.feat_mat.shape[1], self.embedding_size)
        
        self.w = nn.Parameter(torch.ones([self.embedding_size], dtype=torch.float32, device=self.device))
        normal_(self.embedding.weight, std=0.1)
        self.to(device=self.device)
        self.norm_aug_adj = self.generate_aug_graph(model_config['dataset'])
        self.times = model_config.get('times', 0.1)
        # self.aug_num = (self.times) * len(self.feat_mat)

    def update_feat_mat(self):
        row, _ = self.feat_mat.indices()
        edge_values = torch.pow(self.row_sum[row], (self.alpha - 1.) / 2. - 0.5)
        self.feat_mat = torch.sparse.FloatTensor(self.feat_mat.indices(), edge_values, self.feat_mat.shape).coalesce()

    def feat_mat_anneal(self):
        self.alpha *= self.delta
        self.update_feat_mat()

    def generate_graph(self, dataset):
        return LightGCN.generate_graph(self, dataset)
    


    def generate_aug_graph(self, dataset):
        # new graph after adding some interactions
        aug_idx = self.cal_cos_sim()
        aug_adj_mat = generate_aug_daj_mat(dataset, aug_idx)
        degree = np.array(np.sum(aug_adj_mat, axis=1)).squeeze()
        degree = np.maximum(1., degree)
        d_inv = np.power(degree, -0.5) # 累乗
        d_mat = sp.diags(d_inv, format='csr', dtype=np.float32)

        norm_aug_adj = d_mat.dot(aug_adj_mat).dot(d_mat)
        norm_aug_adj = get_sparse_tensor(norm_aug_adj, self.device)
        return norm_aug_adj

    def generate_feat(self, dataset, is_updating=False, ranking_metric=None):
        # return adj matrix with template
        if not is_updating:
            if self.feature_ratio < 1.:  # ランク付後にスライス

                ranked_users, ranked_items = graph_rank_nodes(dataset, ranking_metric)
                core_users = ranked_users[:int(self.n_users * self.feature_ratio)]
                core_items = ranked_items[:int(self.n_items * self.feature_ratio)]
            else:
                core_users = np.arange(self.n_users, dtype=np.int64)
                core_items = np.arange(self.n_items, dtype=np.int64)

            user_map = dict()
            for idx, user in enumerate(core_users):
                user_map[user] = idx
            item_map = dict()
            for idx, item in enumerate(core_items):
                item_map[item] = idx
        else:
            user_map = self.user_map
            item_map = self.item_map

        user_dim, item_dim = len(user_map), len(item_map)
        indices = []
        for user, item in dataset.train_array:
            if item in item_map:
                indices.append([user, user_dim + item_map[item]])
            if user in user_map:
                indices.append([self.n_users + item, user_map[user]])
        for user in range(self.n_users):
            indices.append([user, user_dim + item_dim])
        for item in range(self.n_items):
            indices.append([self.n_users + item, user_dim + item_dim + 1])
        feat = sp.coo_matrix((np.ones((len(indices),)), np.array(indices).T),
                             shape=(self.n_users + self.n_items, user_dim + item_dim + 2), dtype=np.float32).tocsr()
        row_sum = torch.tensor(np.array(np.sum(feat, axis=1)).squeeze(), dtype=torch.float32, device=self.device)
        feat = get_sparse_tensor(feat, self.device)
        return feat, user_map, item_map, row_sum

    def inductive_rep_layer(self, feat_mat):
        # generate embedding by using template
        padding_tensor = torch.empty([max(self.feat_mat.shape) - self.feat_mat.shape[1], self.embedding_size],
                                     dtype=torch.float32, device=self.device)
        padding_features = torch.cat([self.embedding.weight, padding_tensor], dim=0)

        row, column = feat_mat.indices()
        g = dgl.graph((column, row), num_nodes=max(self.feat_mat.shape), device=self.device)
        x = dgl.ops.gspmm(g, 'mul', 'sum', lhs_data=padding_features, rhs_data=feat_mat.values())
        x = x[:self.feat_mat.shape[0], :]
        return x

    def get_def_rep(self):
        # generate final embedding
        feat_mat = NGCF.dropout_sp_mat(self, self.feat_mat)
        representations = self.inductive_rep_layer(feat_mat)

        all_layer_rep = [representations]
        row, column = self.norm_adj.indices()
        g = dgl.graph((column, row), num_nodes=self.norm_adj.shape[0], device=self.device)
        for _ in range(self.n_layers):
            representations = dgl.ops.gspmm(g, 'mul', 'sum', lhs_data=representations, rhs_data=self.norm_adj.values())
            all_layer_rep.append(representations)
        all_layer_rep = torch.stack(all_layer_rep, dim=0)
        final_rep = all_layer_rep.mean(dim=0)
        return final_rep

    def get_aug_rep(self, norm_aug_adj):
        # generate final embedding on aug-graph
        feat_mat = NGCF.dropout_sp_mat(self, self.feat_mat)
        representations = self.inductive_rep_layer(feat_mat)
        
        all_layer_rep = [representations]
        row, column = norm_aug_adj.indices()
        g = dgl.graph((column, row), num_nodes=norm_aug_adj.shape[0], device=self.device)
        for _ in range(self.n_layers):
            representations = dgl.ops.gspmm(g, 'mul', 'sum', lhs_data=representations, rhs_data=norm_aug_adj.values())
            all_layer_rep.append(representations)
        all_layer_rep = torch.stack(all_layer_rep, dim=0)
        final_rep = all_layer_rep.mean(dim=0)
        return final_rep

    def cal_cos_sim(self):
        # calculate cosine similarity with user embeddings and item embeddings (on CPU)
        rep = self.get_def_rep()
        all_users_r = rep[:self.n_users, :]
        all_items_r = rep[self.n_users:, :]
        
        all_users_r = all_users_r.to('cpu').detach().numpy().copy()
        all_items_r = all_items_r.to('cpu').detach().numpy().copy()
        
        x = cosine_similarity(all_users_r, all_items_r)
        cos_mat = torch.from_numpy(x.astype(np.float32)).clone()
        cos_mat.to(device=self.device)
        cos_sim = torch.reshape(cos_mat, (1, -1))
        _, idx = torch.topk(cos_sim, self.aug_num)
        #idx = idx.tolist()S
        aug_idx = [[int(torch.div(idx[0][i], self.n_items, rounding_mode='floor')), (int(torch.fmod(idx[0][i], self.n_items)))] for i in range(self.aug_num)]
        
        return aug_idx  # return list [user_id, item_id]
    
    def cal_cos_sim_v2(self):
        # calculate cosine similarity with user embeddings and item embeddings (on GPU)
        rep = self.get_def_rep()
        all_users_r = rep[:self.n_users, :]
        all_items_r = rep[self.n_users:, :]
        
        cos_sim = pairwise_cosine_similarity(all_users_r, all_items_r)
        cos_sim = torch.reshape(cos_sim, (1, -1))
        _, idx = torch.topk(cos_sim, self.aug_num)
        aug_idx = [[int(torch.div(idx[0][i], self.n_items, rounding_mode='floor')), (int(torch.fmod(idx[0][i], self.n_items)))] for i in range(self.aug_num)]

        return aug_idx  # return list [user_id, item_id]


    def cal_loss(self, users_r, aug_users_r):
        # calcrate ssl-loss(InfoNCE)
        loss = InfoNCE(negative_mode='paired')
        
        query = users_r / self.taugh
        positive_key = aug_users_r
        negative_keys = aug_users_r.unsqueeze(1)
        
        contrasive_loss = loss(query, positive_key, negative_keys)
        return contrasive_loss

    def bpr_forward(self, users, pos_items, neg_items):
        # 普通の埋め込み
        rep = self.get_def_rep()
        users_r = rep[users, :]
        pos_items_r, neg_items_r = rep[self.n_users + pos_items, :], rep[self.n_users + neg_items, :]
        # AUG後の埋め込み
        
        aug_rep = self.get_aug_rep(self.norm_aug_adj)
        aug_users_r = aug_rep[users, :]
        l2_norm_sq = torch.norm(users_r, p=2, dim=1) ** 2 + torch.norm(pos_items_r, p=2, dim=1) ** 2 \
                     + torch.norm(neg_items_r, p=2, dim=1) ** 2
        # Contrasive loss
        contrasive_loss = self.cal_loss(users_r, aug_users_r)
        return users_r, pos_items_r, neg_items_r, l2_norm_sq, contrasive_loss



    def predict(self, users):
        rep = self.get_def_rep()
        users_r = rep[users, :]
        all_items_r = rep[self.n_users:, :]
        scores = torch.mm(users_r, all_items_r.t())
        return scores
    
    def save(self, path):
        params = {'sate_dict': self.state_dict(), 'user_map': self.user_map,
                  'item_map': self.item_map, 'alpha': self.alpha}
        torch.save(params, path)

    def load(self, path):
        params = torch.load(path, map_location=self.device)
        self.load_state_dict(params['sate_dict'])
        self.user_map = params['user_map']
        self.item_map = params['item_map']
        self.alpha = params['alpha']
        self.feat_mat, _, _, self.row_sum = self.generate_feat(self.config['dataset'], is_updating=True)
        self.update_feat_mat()

SyntaxError: EOF while scanning triple-quoted string literal (1017545651.py, line 218)

In [15]:
def update_feat_mat():
    row, _ = feat_mat.indices()
    edge_values = torch.pow(row_sum[row], (alpha - 1.) / 2. - 0.5)
    feat_mat = torch.sparse.FloatTensor(feat_mat.indices(), edge_values, feat_mat.shape).coalesce()

def feat_mat_anneal():
    alpha *= delta
    self.update_feat_mat()


def generate_graph( dataset):
    adj_mat = generate_daj_mat(dataset)
    degree = np.array(np.sum(adj_mat, axis=1)).squeeze()
    degree = np.maximum(1., degree)
    d_inv = np.power(degree, -0.5) # 累乗
    d_mat = sp.diags(d_inv, format='csr', dtype=np.float32)

    norm_adj = d_mat.dot(adj_mat).dot(d_mat)
    norm_adj = get_sparse_tensor(norm_adj, device)
    return norm_adj



def generate_aug_graph(dataset):
    # new graph after adding some interactions
    aug_idx = cal_cos_sim()
    aug_adj_mat = generate_aug_daj_mat(dataset, aug_idx)
    degree = np.array(np.sum(aug_adj_mat, axis=1)).squeeze()
    degree = np.maximum(1., degree)
    d_inv = np.power(degree, -0.5) # 累乗
    d_mat = sp.diags(d_inv, format='csr', dtype=np.float32)

    norm_aug_adj = d_mat.dot(aug_adj_mat).dot(d_mat)
    norm_aug_adj = get_sparse_tensor(norm_aug_adj, device)
    return norm_aug_adj

def generate_feat(dataset, is_updating=False, ranking_metric=None):
    # return adj matrix with template
    if not is_updating:
        if feature_ratio < 1.:  # ランク付後にスライス

            ranked_users, ranked_items = graph_rank_nodes(dataset, ranking_metric)
            core_users = ranked_users[:int(n_users * feature_ratio)]
            core_items = ranked_items[:int(n_items * feature_ratio)]
        else:
            core_users = np.arange(n_users, dtype=np.int64)
            core_items = np.arange(n_items, dtype=np.int64)

        user_map = dict()
        for idx, user in enumerate(core_users):
            user_map[user] = idx
        item_map = dict()
        for idx, item in enumerate(core_items):
            item_map[item] = idx
    else:
        user_map = user_map
        item_map = item_map

    user_dim, item_dim = len(user_map), len(item_map)
    indices = []
    for user, item in dataset.train_array:
        if item in item_map:
            indices.append([user, user_dim + item_map[item]])
        if user in user_map:
            indices.append([n_users + item, user_map[user]])
    for user in range(n_users):
        indices.append([user, user_dim + item_dim])
    for item in range(n_items):
        indices.append([n_users + item, user_dim + item_dim + 1])
    feat = sp.coo_matrix((np.ones((len(indices),)), np.array(indices).T),
                         shape=(n_users + n_items, user_dim + item_dim + 2), dtype=np.float32).tocsr()
    row_sum = torch.tensor(np.array(np.sum(feat, axis=1)).squeeze(), dtype=torch.float32, device=device)
    feat = get_sparse_tensor(feat, device)
    return feat, user_map, item_map, row_sum

def inductive_rep_layer(feat_mat):
    # generate embedding by using template
    padding_tensor = torch.empty([max(feat_mat.shape) - feat_mat.shape[1], embedding_size],
                                 dtype=torch.float32, device=device)
    padding_features = torch.cat([embedding.weight, padding_tensor], dim=0)

    row, column = feat_mat.indices()
    g = dgl.graph((column, row), num_nodes=max(feat_mat.shape), device=device)
    x = dgl.ops.gspmm(g, 'mul', 'sum', lhs_data=padding_features, rhs_data=feat_mat.values())
    x = x[:feat_mat.shape[0], :]
    return x

def get_def_rep():
    # generate final embedding
    #feat_mat = NGCF.dropout_sp_mat(feat_mat)
    representations = inductive_rep_layer(feat_mat)

    all_layer_rep = [representations]
    row, column = norm_adj.indices()
    g = dgl.graph((column, row), num_nodes=norm_adj.shape[0], device=device)
    for _ in range(n_layers):
        representations = dgl.ops.gspmm(g, 'mul', 'sum', lhs_data=representations, rhs_data=norm_adj.values())
        all_layer_rep.append(representations)
    all_layer_rep = torch.stack(all_layer_rep, dim=0)
    final_rep = all_layer_rep.mean(dim=0)
    return final_rep

def get_aug_rep(norm_aug_adj):
    # generate final embedding on aug-graph
    #feat_mat = NGCF.dropout_sp_mat(self, self.feat_mat)
    representations = inductive_rep_layer(feat_mat)

    all_layer_rep = [representations]
    row, column = norm_aug_adj.indices()
    g = dgl.graph((column, row), num_nodes=norm_aug_adj.shape[0], device=device)
    for _ in range(n_layers):
        representations = dgl.ops.gspmm(g, 'mul', 'sum', lhs_data=representations, rhs_data=norm_aug_adj.values())
        all_layer_rep.append(representations)
    all_layer_rep = torch.stack(all_layer_rep, dim=0)
    final_rep = all_layer_rep.mean(dim=0)
    return final_rep

def cal_cos_sim():
    # calculate cosine similarity with user embeddings and item embeddings (on CPU)
    rep = get_def_rep()
    all_users_r = rep[:n_users, :]
    all_items_r = rep[n_users:, :]

    all_users_r = all_users_r.to('cpu').detach().numpy().copy()
    all_items_r = all_items_r.to('cpu').detach().numpy().copy()

    x = cosine_similarity(all_users_r, all_items_r)
    cos_mat = torch.from_numpy(x.astype(np.float32)).clone()
    cos_mat.to(device=device)
    cos_sim = torch.reshape(cos_mat, (1, -1))
    _, idx = torch.topk(cos_sim, aug_num)
    #idx = idx.tolist()S
    aug_idx = [[int(torch.div(idx[0][i], n_items, rounding_mode='floor')), (int(torch.fmod(idx[0][i], n_items)))] for i in range(aug_num)]

    return aug_idx  # return list [user_id, item_id]

def cal_cos_sim_v2():
    # calculate cosine similarity with user embeddings and item embeddings (on GPU)
    rep = get_def_rep()
    all_users_r = rep[:n_users, :]
    all_items_r = rep[n_users:, :]

    cos_sim = pairwise_cosine_similarity(all_users_r, all_items_r)
    cos_sim = torch.reshape(cos_sim, (1, -1))
    _, idx = torch.topk(cos_sim, aug_num)
    aug_idx = [[int(torch.div(idx[0][i], n_items, rounding_mode='floor')), (int(torch.fmod(idx[0][i], n_items)))] for i in range(aug_num)]

    return aug_idx  # return list [user_id, item_id]


def cal_loss(users_r, aug_users_r):
    # calcrate ssl-loss(InfoNCE)
    loss = InfoNCE(negative_mode='unpaired')

    query = users_r / taugh
    positive_key = aug_users_r
    negative_keys = aug_users_r

    contrasive_loss = loss(query, positive_key, negative_keys)
    return contrasive_loss

def bpr_forward(self, users, pos_items, neg_items):
    # 普通の埋め込み
    rep = get_def_rep()
    users_r = rep[users, :]
    pos_items_r, neg_items_r = rep[n_users + pos_items, :], rep[n_users + neg_items, :]
    # AUG後の埋め込み

    aug_rep = get_aug_rep(norm_aug_adj)
    aug_users_r = aug_rep[users, :]
    l2_norm_sq = torch.norm(users_r, p=2, dim=1) ** 2 + torch.norm(pos_items_r, p=2, dim=1) ** 2 \
                 + torch.norm(neg_items_r, p=2, dim=1) ** 2
    # Contrasive loss
    contrasive_loss = cal_loss(users_r, aug_users_r)
    return users_r, pos_items_r, neg_items_r, l2_norm_sq, contrasive_loss


In [16]:
def get_sparse_tensor(mat, device):
    coo = mat.tocoo()
    indexes = np.stack([coo.row, coo.col], axis=0)
    indexes = torch.tensor(indexes, dtype=torch.int64, device=device)
    data = torch.tensor(coo.data, dtype=torch.float32, device=device)
    sp_tensor = torch.sparse.FloatTensor(indexes, data, torch.Size(coo.shape)).coalesce()
    return sp_tensor


def generate_daj_mat(dataset): # Sparseなadj
    train_array = np.array(dataset.train_array)
    users, items = train_array[:, 0], train_array[:, 1]
    row = np.concatenate([users, items + dataset.n_users], axis=0)
    column = np.concatenate([items + dataset.n_users, users], axis=0)
    adj_mat = sp.coo_matrix((np.ones(row.shape), np.stack([row, column], axis=0)),
                            shape=(dataset.n_users + dataset.n_items, dataset.n_users + dataset.n_items),
                            dtype=np.float32).tocsr()
    return adj_mat
def generate_aug_daj_mat(dataset, aug_idx): # Sparseなadj
    # generate adj matrix on aug-graph
    train_array = dataset.train_array
    train_array.extend(aug_idx)
    train_array = np.array(train_array)
    users, items = train_array[:, 0], train_array[:, 1]
    row = np.concatenate([users, items + dataset.n_users], axis=0)
    column = np.concatenate([items + dataset.n_users, users], axis=0)
    adj_mat = sp.coo_matrix((np.ones(row.shape), np.stack([row, column], axis=0)),
                            shape=(dataset.n_users + dataset.n_items, dataset.n_users + dataset.n_items),
                            dtype=np.float32).tocsr()
    return adj_mat

def graph_rank_nodes(dataset, ranking_metric):
    adj_mat = generate_daj_mat(dataset)
    if ranking_metric == 'degree':
        user_metrics = np.array(np.sum(adj_mat[:dataset.n_users, :], axis=1)).squeeze()
        item_metrics = np.array(np.sum(adj_mat[dataset.n_users:, :], axis=1)).squeeze()
    elif ranking_metric == 'greedy' or ranking_metric == 'sort':
        '''
        # This is for theoretical analysis.
        part_adj = adj_mat[:dataset.n_users, dataset.n_users:]
        part_adj_tensor = get_sparse_tensor(part_adj, 'cpu')
        with torch.no_grad():
            u, s, v = torch.svd_lowrank(part_adj_tensor, 64)
            u, v = u.numpy(), v.numpy()
        user_metrics = greedy_or_sort(part_adj, u, ranking_metric, dataset.device)
        item_metrics = greedy_or_sort(part_adj.T, v, ranking_metric, dataset.device)
        '''
        normalized_adj_mat = normalize(adj_mat, axis=1, norm='l1')
        user_metrics = np.array(np.sum(normalized_adj_mat[:, :dataset.n_users], axis=0)).squeeze()
        item_metrics = np.array(np.sum(normalized_adj_mat[:, dataset.n_users:], axis=0)).squeeze()
    elif ranking_metric == 'page_rank':
        g = nx.Graph()
        g.add_edges_from(np.array(np.nonzero(adj_mat)).T)
        pr = nx.pagerank(g)
        pr = np.array([pr[i] for i in range(dataset.n_users + dataset.n_items)])
        user_metrics, item_metrics = pr[:dataset.n_users], pr[dataset.n_users:]
    else:
        return None
    ranked_users = np.argsort(user_metrics)[::-1].copy()
    ranked_items = np.argsort(item_metrics)[::-1].copy()
    return ranked_users, ranked_items



In [17]:
config = model_config
name = model_config['name']
device = model_config['device']
n_users = dataset.n_users
n_items = dataset.n_items
trainable = True

In [18]:
embedding_size = model_config['embedding_size']
n_layers = model_config['n_layers']
dropout = model_config['dropout']
feature_ratio = model_config['feature_ratio']  # Template%
norm_adj = generate_graph(dataset)

alpha = 1.
delta = model_config.get('delta', 0.99)
taugh = model_config.get('taugh', 0.2)
aug_num = model_config['aug_num']
# self.temper = model_config['temper']


In [19]:
feat_mat, user_map, item_map, row_sum = \
    generate_feat(dataset,
                       ranking_metric=model_config.get('ranking_metric', 'sort'))
#update_feat_mat()
# self.norm_aug_adj = enerate_aug_graph(model_config['dataset'] )
embedding = nn.Embedding(feat_mat.shape[1], embedding_size)

w = nn.Parameter(torch.ones([embedding_size], dtype=torch.float32, device=device))
normal_(embedding.weight, std=0.1)
embedding.to(device=device)
norm_aug_adj = generate_aug_graph(dataset)
times = model_config.get('times', 0.1)
# self.aug_num = (self.times) * len(self.feat_mat)

In [20]:
feat_mat

tensor(indices=tensor([[    0,     0,     0,  ..., 70845, 70845, 70845],
                       [17914, 17918, 17922,  ..., 16861, 17305, 42507]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       device='cuda:0', size=(70846, 42508), nnz=2028052,
       layout=torch.sparse_coo)

In [21]:
norm_adj

tensor(indices=tensor([[    0,     0,     0,  ..., 70845, 70845, 70845],
                       [29858, 29859, 29861,  ..., 26841, 28266, 29001]]),
       values=tensor([0.0163, 0.0057, 0.0119,  ..., 0.0173, 0.0541, 0.0358]),
       device='cuda:0', size=(70846, 70846), nnz=2286148,
       layout=torch.sparse_coo)

In [22]:
norm_aug_adj

tensor(indices=tensor([[    0,     0,     0,  ..., 70845, 70845, 70845],
                       [29858, 29859, 29861,  ..., 26860, 28266, 29001]]),
       values=tensor([0.0102, 0.0048, 0.0096,  ..., 0.0570, 0.0489, 0.0333]),
       device='cuda:0', size=(70846, 70846), nnz=3142568,
       layout=torch.sparse_coo)

In [15]:
aug_idx

NameError: name 'aug_idx' is not defined

In [6]:
dataloader = DataLoader(dataset, batch_size=trainer_config['batch_size'],
                                     num_workers=trainer_config['dataloader_num_workers'])



In [9]:
for batch_data in dataloader:
    #start_time1 = time.time()
    inputs = batch_data[:, 0, :].to(device=device, dtype=torch.int64)
    users, pos_items, neg_items = inputs[:, 0],  inputs[:, 1],  inputs[:, 2]
    break

In [ ]:
all_r = get_def_rep()

In [11]:
pos_items.shape

torch.Size([2048])

In [ ]:
aug_r = get_aug_rep(norm_aug_adj)

In [16]:
"""for batch_data in dataloader:
    inputs = batch_data[:, 0, :].to(device=device, dtype=torch.int64)
    users, pos_items, neg_items = inputs[:, 0],  inputs[:, 1],  inputs[:, 2]
    user_r = all_r[users, :]
    print(user_r.sort())
    aug_user_r = aug_r[users, :]
    loss = cal_loss(user_r, aug_user_r)
    print(loss)

SyntaxError: EOL while scanning string literal (3381987239.py, line 1)

In [7]:
inputs = bach_data[:, 0, :].to(device=device, dtype=torch.int64)
users, pos_items, neg_items = inputs[:, 0],  inputs[:, 1],  inputs[:, 2]

NameError: name 'bach_data' is not defined

In [8]:
user_r = all_r[users, :]
aug_user_r = aug_r[users, :]

NameError: name 'all_r' is not defined

In [45]:
user_r.size()

torch.Size([296, 64])

In [46]:
aug_user_r.size()

torch.Size([296, 64])

In [ ]:
loss = cal_loss(user_r, aug_user_r)
loss

In [ ]:
dataloader

In [ ]:
users

In [ ]:
embedding(users)


In [ ]:
embedding(pos_items + len(user_map))

In [25]:
pos_items_r, neg_items_r = all_r[n_users + pos_items, :], all_r[n_users + neg_items, :]

In [26]:
#users_r, pos_items_r, neg_items_r, l2_norm_sq, contrastive_loss = self.model.bpr_forward(users, pos_items, neg_items)
pos_scores = torch.sum(user_r * pos_items_r, dim=1)
neg_scores = torch.sum(user_r * neg_items_r, dim=1)
bpr_loss = F.softplus(neg_scores - pos_scores).mean()

In [27]:
bpr_loss

tensor(0.4619, device='cuda:0', grad_fn=<MeanBackward0>)

In [28]:
losses = bpr_loss + loss

In [29]:
losses

tensor(1.1551, device='cuda:0', grad_fn=<AddBackward0>)

In [24]:
ranking_metric = 'degree'
ranked_users, ranked_items = graph_rank_nodes(dataset, ranking_metric)

In [25]:
ranked_users

array([  745,  2751,   820, ..., 13678, 20115, 18270])

In [29]:
all_r[ranked_users, :]

tensor([[ 1.0823,  0.3671,  0.0626,  ..., -0.7633,  0.0223, -1.5940],
        [ 1.3623,  1.1896,  0.0042,  ..., -2.2789,  2.1133,  0.4617],
        [ 1.6749, -0.0745,  0.7189,  ..., -0.6808,  0.8492, -0.6798],
        ...,
        [ 0.0543,  0.1464, -0.0403,  ..., -0.0886,  0.1143,  0.1149],
        [-0.0938, -0.2187,  0.0548,  ..., -0.0205,  0.1147,  0.0090],
        [ 0.0404, -0.0958,  0.0739,  ..., -0.0384, -0.0439,  0.0160]],
       device='cuda:0', grad_fn=<IndexBackward0>)

In [40]:
all_r[n_users+ranked_items, :].size()

torch.Size([40988, 64])

In [37]:
len(ranked_items)

40988

In [1]:
import numpy as np

In [2]:
mylist = np.array([1, 2, 3, 4, 5, 6, 7, 8])

In [6]:
mylist2 = mylist.reshape(2, 4)
mylist2

array([[1, 2, 3, 4],
       [5, 6, 7, 8]])

In [4]:
mylist3 = mylist2.reshape(1, -1)

In [5]:
mylist3

array([[1, 2, 3, 4, 5, 6, 7, 8]])